# Andrew Kim

# ADS 509-01-SU23: Applied Text Mining

# Assignment 4.1: Text Classification Model

# 6/5/2023

## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [1]:
!pip install tokenization

In [2]:
!pip install functions

In [3]:
import sqlite3
import nltk
import random
import numpy as np
import collections
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [4]:
# Additional packages imported
import pandas as pd
import tokenize
import re
import string
from string import punctuation
import collections
from collections import Counter, defaultdict
from nltk.corpus import stopwords

In [5]:
# Import 2020_Conventions.db
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

Note: A bit of reversal from last week's strategy. To deal with stopwords at the end of a sentence (e.g. He tossed it.), I removed punctuation except for apostrophes to save the contractions for stopword removal. This seemed to line up with the assertion a few chunks down. 

In [6]:
# First, modify 2020_Conventions.db to remove stopwords, punctuation, and return tokens
sw = stopwords.words("english")
punctuation = set(punctuation)
tw_punct = punctuation - {"#", "'"}

def remove_stop(text, sw=sw) :
    return([ch for ch in text if ch not in sw])

def remove_punctuation(text, punct_set=tw_punct) : 
   text = [''.join(ch for ch in word if ch not in punct_set)
           for word in text]
   
   return(text)

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    text = text.strip().split()
    return(text)

In [7]:
convention_data = []

# fill this list up with items that are themselves lists. 
# The first element in the sublist should be the cleaned and tokenized
# text in a single string. 
# The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party
                            FROM conventions;
                            ''')

for row in query_results :

    # Clean the text from the data set
    text = row[0]
    text_lc = text.lower()
    text_tok = tokenize(text_lc)
    text_punct = remove_punctuation(text_tok)
    text_stop = remove_stop(text_punct)
    text_clean = " ".join(text_stop)
    
    # Store the results in convention_list
    pparty = row[1]
    plist = [text_clean, pparty]
    
    convention_data.append(plist)

Let's look at some random entries and see if they look right. 

In [8]:
random.choices(convention_data,k=10)

[['beau hunter said “no mom mommy mommy died mom” along came ashley dr',
  'Democratic'],
 ['connecticut', 'Republican'],
 ['could use little help right seems like get one step forward two steps back',
  'Democratic'],
 ['young people watching don’t give america despite flaws problems we’ve come far still equal country opportunities previous generations could never imagined there’s lot heartbreak america truth many things broken pandemic saying goes world breaks everyone afterward many strong broken places that’s joe biden knows keep going unify lead he’s done family country come november we’re strong together we’ll heal together we’ll redeem soul',
  'Democratic'],
 ['spent five years working report trump administration started remove mention humans cause climate change dr',
  'Democratic'],
 ['herby declare', 'Republican'],
 ['you’re kind forgotten right president always bragging stock market sort leaves lot small manufacturing companies behind',
  'Democratic'],
 ['radical left take

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [9]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [10]:
def conv_features(text,fw = feature_words) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    text_tok = text.split()
    text_feat_words = [ch for ch in text_tok if ch in feature_words]
    ret_dict = dict.fromkeys(text_feat_words, True)
    
    return(ret_dict)

In [11]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [12]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [13]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [14]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [15]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Response: __Based off the classifier and from a general perspective, the list that contains the most informative features correlates to today's society and the current issues that impose potential threats or future uncertainties within everyday life. These are features that have been and continue to be evoked or mentioned between either or both parties.__

__From an analytical perspective, I found it both interesting and odd that 92% of the most informative features leans more with the Republican Party, with the Democratic Party only receiving the terms "votes" and "climate" as their key importative features. While it would make sense that the term "climate" would be emphasized more from the Democratic Party, I found it odd that the term "liberal" leans more towards the Republican Party when I initially perceived "liberal" to be more emphasized from the Democratic Party. In general, I sense a huge disparity or imbalance within each feature being more directed towards the Republican Party than the Democratic Party. Therefore, the possibility of the data being imbalanced or even biased cannot be ruled out.__


## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [16]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [17]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [18]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

for row in results :

    # Like the first process, clean the text within the twitter feed. 
    tweet = row[2].decode('utf-8')
    tweet_lc = text.lower()
    tweet_tok = tokenize(text_lc)
    tweet_punct = remove_punctuation(tweet_tok)
    tweet_stop = remove_stop(tweet_punct)
    tweet_clean = " ".join(tweet_stop)
    tweet_nourl = re.sub(r'http\S+', '', tweet_clean).strip()
    
    # Create list of cleaned data to store to original data file
    tparty = row[1]
    tlist = [tweet_nourl, tparty]
    
    tweet_data.append(tlist)

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [19]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [20]:
for tweet, party in tweet_data_sample :
    features = conv_features(text=tweet)
    estimated_party = classifier.classify(featureset=features)
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")

Here's our (cleaned) tweet: concludes convention program evening stand recess tomorrow evening related transcripts iowa gov kim reynolds press conference transcript september 16 • 1 hour ago press secretary kayleigh mcenany white house press conference transcript september 16 • 1 hour ago doj press conference transcript september 16 charges 5 chinese nationals • 2 hours ago stay updated get weekly digest week’s important transcripts inbox it’s news without news please enable javascript browser complete form email  kind transcripts want read submit transcription overview works faq mobile app captions overview works faq caption converter subtitles translation overview certified translation business translation faq request quote company press careers freelancers blog api get touch 222 kearny st 8th floor san francisco ca 94108 contact us 8883690701 supportrevcom © revcom reviews  terms  privacy stay updated get weekly digest week’s important transcripts inbox it’s news without news please

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [21]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0

num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
    features = conv_features(text=tweet)
    
    # get the estimated party
    estimated_party = classifier.classify(featureset=features)

    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [22]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 4278, 'Democratic': 0}),
             'Democratic': defaultdict(int,
                         {'Republican': 5724, 'Democratic': 0})})

### Reflections

Response: __The results here appear identical from the first section that generated the most informative features, with the Republican Party being more favored than the Democratic Party. Therefore, this validates a huge presence of potential bias or imbalance within the given data that needs to be addressed.__